<a href="https://colab.research.google.com/github/sodonne6/Data_Augmentation_for_AVSR/blob/main/Av_Hubert_first_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install -U condacolab
import condacolab
condacolab.install() #restarts runtime

✨🍰✨ Everything looks OK!


In [ ]:
#create and activate python 3.8 env
%%bash
eval "$(conda shell.bash hook)"
conda env remove -n av_hubert || true # Remove the environment if it exists
conda create -n av_hubert python=3.8 -y # Add -y to automatically confirm
conda activate av_hubert

# clone and install dependencies inside env
git clone https://github.com/facebookresearch/av_hubert.git /content/av_hubert || true # Add || true to prevent exit on existing directory
cd /content/av_hubert
git submodule init
git submodule update

#show content for sanity check
pwd
ls -lah

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local/envs/av_hubert

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       hda65f42_8         254 KB  conda-forge
    ca-certificates-2025.10.5  |       hbd8a1cb_0         152 KB  conda-forge
    ld_impl_linux-64-2.44      |       ha97dd6f_2         730 KB  conda-forge
    libffi-3.4.6               |       h2dba641_1          56 KB  conda-forge
    libgcc-15.2.0              |       h767d61c_5         803 KB  conda-forge
    libgcc-ng-15.2.0           |       h69a702a_5          29 KB  conda-forge
    libgomp-15.2.0             |       h767d61c_5         438 KB  conda-forge
    liblzma-5.8.1              |       hb9d3cd8_2         110 KB  conda-forge
    liblzma-devel-5.8.1        |


EnvironmentLocationNotFound: Not a conda environment: /usr/local/envs/av_hubert



==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda


Cloning into '/content/av_hubert'...
Submodule 'fairseq' (https://github.com/pytorch/fairseq) registered for path 'fairseq'
Cloning into '/content/av_hubert/fairseq'...


In [ ]:
# install deps inside the conda env
%%bash
eval "$(conda shell.bash hook)"
conda activate av_hubert
cd /content/av_hubert/

# 0) Repo requirements
pip install -r requirements.txt

# 1) Pin NumPy so np.float exists
pip install --no-cache-dir "numpy==1.23.5"

# 2) Pin config stack that works with this fairseq tree
pip uninstall -y omegaconf hydra-core || true
pip install --no-cache-dir omegaconf==2.1.1 hydra-core==1.1.0

# 3) Torch (GPU build via pip cu117 wheels)
conda remove -y pytorch torchvision torchaudio pytorch-cuda || true
pip uninstall -y torch torchvision torchaudio || true
pip install --no-cache-dir \
  torch==1.13.1+cu117 \
  torchvision==0.14.1+cu117 \
  torchaudio==0.13.1 \
  --extra-index-url https://download.pytorch.org/whl/cu117

# 4) Install fairseq from the bundled submodule (editable) **without deps**
cd fairseq
pip install -e . --no-deps
cd ..

# 5) Re-assert the pins in case anything tried to downgrade them (belt & suspenders)
pip install --no-cache-dir --upgrade --upgrade-strategy only-if-needed \
  omegaconf==2.1.1 hydra-core==1.1.0

# 6) Tiny import check
python - << 'PY'
import torch, fairseq, numpy as np
from omegaconf import II, MISSING
import hydra
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())
print("Fairseq import OK")
print("NumPy:", np.__version__)
print("OmegaConf II/MISSING present:", II is not None and MISSING is not None)
print("Hydra:", hydra.__version__)
PY


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 175.6 MB/s eta 0:00:00
  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5865 sha256=7acfb5fac94ec2df705b061d74e32b1a4ee89205a578a7eaf66da470dd788b14
  Stored in directory: /root/.cache/pip/wheels/5b/60/87/28af2605138deac93d162904df42b6fdda1dab9b8757c62aa3
Successfully built python-speech-features
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 53.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
  Preparing metadata (setu


PackagesNotFoundError: The following packages are missing from the target environment:
  - pytorch-cuda
  - torchvision
  - torchaudio
  - pytorch




In [ ]:
#CODE BEGINNING -> MOUNT GOOGLE DRIVE
#NEEDS TO BE DONE EVERYTIME

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title AV-Hubert data and checkpoint paths

#set paths for data and checkpoints
REPO_DIR = '/content/av_hubert'  # repo root (contains 'avhubert/' and 'fairseq/' folders)
DATA_DIR = # your prepared manifests dir
CKPT     = '/content/drive/MyDrive/AVSR_Colab/models/base_vox_iter5.pt'     # pre-trained checkpoint
RUN_DIR  = # experiment folder (resume-safe)

# Modalities: '["audio","video"]' or '["audio"]' if lips are poor
MODALITIES = '["audio","video"]'

# VRAM-friendly defaults
STACK_ORDER_AUDIO = 4     # (16kHz + 25 fps labels) => 26*4=104-dim features
MAX_TOKENS        = 100000 # reduce if you still OOM (e.g., 60000)
UPDATE_FREQ       = 2      # gradient accumulation
NUM_WORKERS       = 1

In [ ]:
## 4) Sanity checks on manifests
import os
req = [f"{DATA_DIR}/train.tsv", f"{DATA_DIR}/valid.tsv", f"{DATA_DIR}/dict.wrd.txt"]
missing = [p for p in req if not os.path.isfile(p)]
assert not missing, f"Missing files: {missing}"
print('Found train/valid/dict ✅')
print('train.tsv head:')
with open(f"{DATA_DIR}/train.tsv") as f:
    for i, ln in zip(range(6), f):
        print(ln.rstrip())

## Load in the pretrained model


In [ ]:
%%bash
eval "$(conda shell.bash hook)"
conda activate av_hubert
cd /content  # stay outside the repo root

# Put the avhubert *subfolder* on PYTHONPATH so top-level imports work
PYTHONPATH=/content/av_hubert/avhubert:$PYTHONPATH \
python -u - << 'PY'
import os, torch, importlib

# 1) Import fairseq first
import fairseq
from fairseq.tasks import TASK_REGISTRY
print("fairseq OK; registry size before:", len(TASK_REGISTRY))

# 2) Force-import the files that perform registration
#    (these use @register_task / @register_model decorators)
importlib.import_module("hubert_pretraining")  # registers 'av_hubert_pretraining'
importlib.import_module("hubert")              # registers the AV-HuBERT model

# 3) Verify registration
from fairseq.tasks import TASK_REGISTRY
print("Contains 'av_hubert_pretraining':", "av_hubert_pretraining" in TASK_REGISTRY)

# 4) Load checkpoint
from fairseq.checkpoint_utils import load_model_ensemble_and_task
ckpt_path = "/content/drive/MyDrive/AVSR_Colab/models/base_vox_iter5.pt"  # adjust if needed
assert os.path.exists(ckpt_path), f"Checkpoint not found: {ckpt_path}"

models, cfg, task = load_model_ensemble_and_task([ckpt_path])
model = models[0]

print("✅ Loaded AV-HuBERT checkpoint")
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("Model:", type(model).__name__, "| Task:", type(task).__name__)
PY


fairseq OK; registry size before: 25
Contains 'av_hubert_pretraining': True
2025-10-06 20:35:20 | INFO | hubert_pretraining | current directory is /content
2025-10-06 20:35:20 | INFO | hubert_pretraining | AVHubertPretrainingTask Config {'_name': 'av_hubert_pretraining', 'data': '/checkpoint/bshi/data/lrs3//video/wav/envox_tsv/', 'labels': ['km'], 'label_dir': '/checkpoint/bshi/data/lrs3//video/hubert/stitch-iters/envox-iter4-l12c2000/', 'label_rate': 25, 'sample_rate': 25, 'normalize': True, 'enable_padding': False, 'max_sample_size': 2000, 'min_sample_size': 5, 'max_trim_sample_size': 400, 'single_target': False, 'random_crop': True, 'pad_audio': False, 'pdb': False, 'stack_order_audio': 4, 'skip_verify': False, 'image_aug': True, 'image_crop_size': 88, 'image_mean': 0.421, 'image_std': 0.165, 'modalities': ['audio', 'video'], 'is_s2s': False, 'tokenizer_bpe_name': None, 'tokenizer_bpe_model': None, 'noise_wav': '/checkpoint/bshi/data/lrs3//audio/noise/tsv/musan-lgall/', 'noise_prob'

# Setup Training and Validation Environment


The Following cells are to set up audio and labels. The audio is provided at 48kHz but AV-Hubert is trained on 16kHz so needs to be downsampled. Then the processed audio and the labels can be put in order and match using a manifest.

In [ ]:
## 5) TRAIN — CTC (default). Re-run this cell to resume (same RUN_DIR)
assert FINETUNE_STYLE == 'ctc', 'Switch FINETUNE_STYLE to ctc or use the Seq2Seq cell below.'
%%bash -s "$REPO_DIR" "$DATA_DIR" "$CKPT" "$RUN_DIR" "$MODALITIES" "$STACK_ORDER_AUDIO" "$MAX_TOKENS" "$UPDATE_FREQ" "$NUM_WORKERS"
set -euo pipefail
REPO_DIR="$1"; DATA_DIR="$2"; CKPT="$3"; RUN_DIR="$4"; MODALITIES="$5"; SOA="$6"; MAXTOK="$7"; UPF="$8"; NW="$9"
export HYDRA_FULL_ERROR=1
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
export PYTHONPATH="${REPO_DIR}/avhubert:${PYTHONPATH:-}"

python -m fairseq_cli.hydra_train \
  --config-dir "${REPO_DIR}/avhubert/conf/finetune" \
  --config-name base_vox_30h \
  common.user_dir="${REPO_DIR}/avhubert" \
  hydra.run.dir="${RUN_DIR}" \
  task.data="${DATA_DIR}" \
  task.label_dir="${DATA_DIR}" \
  task.labels='["ltr"]' \
  task.modalities="${MODALITIES}" \
  task.is_s2s=false \
  task.normalize=true \
  task.image_aug=false \
  task.stack_order_audio="${SOA}" \
  task.max_sample_size=null \
  dataset.train_subset=train \
  dataset.valid_subset=valid \
  dataset.num_workers="${NW}" \
  dataset.max_tokens="${MAXTOK}" \
  model._name=av_hubert_ctc \
  model.w2v_path="${CKPT}" \
  model.freeze_finetune_updates=0 \
  model.decoder_layers=3 \
  +model.w2v_args.task._name=av_hubert_pretraining \
  +model.w2v_args.task.data="${DATA_DIR}" \
  +model.w2v_args.task.label_dir="${DATA_DIR}" \
  +model.w2v_args.task.labels='["ltr"]' \
  +model.w2v_args.task.modalities="${MODALITIES}" \
  +model.w2v_args.task.sample_rate=16000 \
  +model.w2v_args.task.label_rate=25 \
  +model.w2v_args.task.pad_audio=true \
  +model.w2v_args.task.random_crop=false \
  +model.w2v_args.task.single_target=true \
  +model.w2v_args.task.fine_tuning=false \
  +model.w2v_args.task.normalize=true \
  +model.w2v_args.task.image_aug=false \
  +model.w2v_args.task.stack_order_audio="${SOA}" \
  +model.w2v_args.label_rate=25 \
  +model.w2v_args.model._name=av_hubert \
  +model.w2v_args.model.input_modality=audio \
  +model.w2v_args.model.audio_feat_dim=$((26*SOA)) \
  +model.w2v_args.model.label_rate=25 \
  criterion._name=ctc \
  +criterion.zero_infinity=true \
  +criterion.post_process=letter \
  optimization.max_update=150 \
  optimization.lr='[1e-4]' \
  optimization.update_freq="[${UPF}]" \
  optimization.clip_norm=5.0 \
  lr_scheduler.warmup_steps=5 \
  lr_scheduler.hold_steps=5 \
  lr_scheduler.decay_steps=140 \
  checkpoint.save_dir="${RUN_DIR}/checkpoints" \
  checkpoint.save_interval_updates=50 \
  checkpoint.best_checkpoint_metric=loss \
  distributed_training.distributed_world_size=1 \
  common.log_interval=5


CKPT: /content/drive/MyDrive/AVSR_Colab/models/base_vox_iter5.pt
DATA: /content/drive/MyDrive/tcdtimit/volunteers/01M/manifests
RUN : /content/drive/MyDrive/tcdtimit/volunteers/01M/runs/ctc_av_test


Anything below here needs to be organised (deleted/migrate to preprocessing)

In [ ]:
from pathlib import Path
import re

ROOT = Path("/content/drive/MyDrive/tcdtimit")
SPEAKER = "volunteers/01M"

ROI_ROOT  = ROOT/SPEAKER/"Clips/video/straightcam"            # 88x88 mp4s
AUD_ROOT  = ROOT/SPEAKER/"Clips/Audio16k/Audio/straightcam"   # 16k wavs
LAB_ROOT  = ROOT/SPEAKER/"Clips/Labels/straightcam"           # TXT word timings
LRS3_ROOT = ROOT/SPEAKER/"Clips"

FILELIST  = LRS3_ROOT/"file.list"
LABELLIST = LRS3_ROOT/"label.list"

def ids_with_ext(root, ext):
    return {p.relative_to(root).with_suffix('').as_posix() for p in root.rglob(f"*.{ext}")}

def read_sentence_from_lab(path):
    words=[]
    with path.open() as f:
        for line in f:
            parts=line.strip().split()
            if len(parts)<3: continue
            w=parts[2]
            if w.lower() in {"sil","sp","spn","noise"}: continue
            words.append(w.lower())
    sent=" ".join(words)
    sent=re.sub(r"[^a-z']+"," ",sent).strip()
    return sent

roi_l = {i.lower(): i for i in ids_with_ext(ROI_ROOT, "mp4")}
wav_l = {i.lower(): i for i in ids_with_ext(AUD_ROOT, "wav")}
lab_l = {i.lower(): i for i in ids_with_ext(LAB_ROOT, "txt")}

common_lower = sorted(set(roi_l) & set(wav_l) & set(lab_l))

# file.list uses ROI-side id casing (so paths resolve to the mp4s)
with FILELIST.open("w") as f:
    for lid in common_lower:
        f.write(roi_l[lid] + "\n")

# label.list lines: "<id>\t<transcription>"
with LABELLIST.open("w") as g:
    for lid in common_lower:
        id_for_lists = roi_l[lid]
        lab_path = LAB_ROOT / (lab_l[lid] + ".txt")
        g.write(f"{id_for_lists}\t{read_sentence_from_lab(lab_path)}\n")

print(f"wrote {len(common_lower)} ids -> {FILELIST}")
print(f"wrote {len(common_lower)} labels -> {LABELLIST}")

# quick peek
print("sample:")
for lid in common_lower[:5]:
    print(roi_l[lid], "->", (LAB_ROOT/(lab_l[lid]+'.txt')).name)


KeyboardInterrupt: 

In [ ]:
# @title debug label.list creation

from pathlib import Path

ROOT = Path("/content/drive/MyDrive/tcdtimit")
SPEAKER = "volunteers/01M"

ROI_ROOT = ROOT/SPEAKER/"Clips/video/straightcam"
AUD_ROOT = ROOT/SPEAKER/"Clips/Audio16k/Audio/straightcam"
LAB_ROOT = ROOT/SPEAKER/"Clips/Labels/straightcam"

def ids_with_ext(root, ext):
    return {p.relative_to(root).with_suffix('').as_posix() for p in root.rglob(f"*.{ext}")}

roi_ids = sorted(ids_with_ext(ROI_ROOT, "mp4"))
wav_ids = sorted(ids_with_ext(AUD_ROOT, "wav"))
lab_ids = sorted(ids_with_ext(LAB_ROOT, "txt"))

print("counts: ROI", len(roi_ids), "WAV", len(wav_ids), "LAB", len(lab_ids))
print("\nSample ROI ids:", roi_ids[:10])
print("\nSample WAV ids:", wav_ids[:10])
print("\nSample LAB ids:", lab_ids[:10])

# show what matches if we ignore case
roi_l = {i.lower(): i for i in roi_ids}
wav_l = {i.lower(): i for i in wav_ids}
lab_l = {i.lower(): i for i in lab_ids}
common_lower = sorted(set(roi_l) & set(wav_l) & set(lab_l))

print("\nCommon ids (case-insensitive) ->", len(common_lower))
print("First 10:", common_lower[:10])


counts: ROI 98 WAV 98 LAB 98

Sample ROI ids: ['sa1', 'sa2', 'si1004', 'si1092', 'si1174', 'si1175', 'si1178', 'si1266', 'si1447', 'si1453']

Sample WAV ids: ['sa1', 'sa2', 'si1004', 'si1092', 'si1174', 'si1175', 'si1178', 'si1266', 'si1447', 'si1453']

Sample LAB ids: ['SA1', 'SA2', 'SI1004', 'SI1092', 'SI1174', 'SI1175', 'SI1178', 'SI1266', 'SI1447', 'SI1453']

Common ids (case-insensitive) -> 98
First 10: ['sa1', 'sa2', 'si1004', 'si1092', 'si1174', 'si1175', 'si1178', 'si1266', 'si1447', 'si1453']


In [ ]:
# @title make compat symlinks
%%bash
SPEAKER=volunteers/01M
CLIPS="/content/drive/MyDrive/tcdtimit/${SPEAKER}/Clips"

# ROI MP4s live in video/straightcam → link to video/
mkdir -p "${CLIPS}"
rm -f "${CLIPS}/video" "${CLIPS}/audio"
ln -s "${CLIPS}/video/straightcam" "${CLIPS}/video"

# 16k WAVs live in Audio16k/Audio/straightcam → link to audio/
ln -s "${CLIPS}/Audio16k/Audio/straightcam" "${CLIPS}/audio"

# sanity
ls -lah "${CLIPS}/video" | head
ls -lah "${CLIPS}/audio" | head


total 2.4M
-rw------- 1 root root  27K Oct  6 19:42 sa1.mp4
-rw------- 1 root root  25K Oct  6 19:42 sa2.mp4
-rw------- 1 root root  32K Oct  6 19:42 si1004.mp4
-rw------- 1 root root  19K Oct  6 19:42 si1092.mp4
-rw------- 1 root root  23K Oct  6 19:42 si1174.mp4
-rw------- 1 root root  30K Oct  6 19:42 si1175.mp4
-rw------- 1 root root  31K Oct  6 19:42 si1178.mp4
-rw------- 1 root root  20K Oct  6 19:42 si1266.mp4
-rw------- 1 root root  33K Oct  6 19:42 si1447.mp4
total 15M
-rw------- 1 root root 163K Oct  6 19:59 sa1.wav
-rw------- 1 root root 149K Oct  6 19:59 sa2.wav
-rw------- 1 root root 203K Oct  6 19:59 si1004.wav
-rw------- 1 root root 116K Oct  6 19:59 si1092.wav
-rw------- 1 root root 143K Oct  6 19:59 si1174.wav
-rw------- 1 root root 195K Oct  6 19:59 si1175.wav
-rw------- 1 root root 193K Oct  6 19:59 si1178.wav
-rw------- 1 root root 137K Oct  6 19:59 si1266.wav
-rw------- 1 root root 203K Oct  6 19:59 si1447.wav


rm: cannot remove '/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/video': Is a directory
rm: cannot remove '/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/audio': Is a directory
ln: failed to create symbolic link '/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/video/straightcam': File exists


In [ ]:
# @title ==== Prep env + paths, then run AV-HuBERT scripts ====
%%bash
set -e

# 0) settings
SPEAKER=volunteers/01M
CLIPS="/content/drive/MyDrive/tcdtimit/${SPEAKER}/Clips"
REPO_DIR=/content/av_hubert

# 1) python deps (cv2, numpy, sentencepiece)
pip -q install "opencv-python-headless==4.8.*" "numpy==1.26.*" sentencepiece

# 2) Make LRS3-like flat dirs (NO symlinks; copy files into flat video/ and audio/)
mkdir -p "${CLIPS}/video" "${CLIPS}/audio"

# Copy ROI mp4s: .../video/straightcam/*.mp4 -> .../video/<id>.mp4
find "${CLIPS}/video/straightcam" -maxdepth 1 -type f -name "*.mp4" -print0 | while IFS= read -r -d '' f; do
  bn="$(basename "$f")"
  [ -f "${CLIPS}/video/${bn}" ] || cp "$f" "${CLIPS}/video/${bn}"
done

# Copy 16k wavs: .../Audio16k/Audio/straightcam/*.wav -> .../audio/<id>.wav
find "${CLIPS}/Audio16k/Audio/straightcam" -maxdepth 1 -type f -name "*.wav" -print0 | while IFS= read -r -d '' f; do
  bn="$(basename "$f")"
  [ -f "${CLIPS}/audio/${bn}" ] || cp "$f" "${CLIPS}/audio/${bn}"
done

echo "Counts after copy:"
echo " video:" $(find "${CLIPS}/video" -maxdepth 1 -name "*.mp4" | wc -l)
echo " audio:" $(find "${CLIPS}/audio" -maxdepth 1 -name "*.wav" | wc -l)

# 3) Sanity: first 5 IDs in file.list
echo "Sample IDs:"
head -n 5 "${CLIPS}/file.list" || true

# 4) Run the scripts
python "${REPO_DIR}/avhubert/preparation/count_frames.py" \
  --root "$CLIPS" \
  --manifest "$CLIPS/file.list" \
  --nshard 1 --rank 0

# 5) Small valid split (adjust as you like)
head -n 10 "$CLIPS/file.list" > "$CLIPS/valid_ids.txt"

# 6) Build manifests (+ vocab)
python "${REPO_DIR}/avhubert/preparation/lrs3_manifest.py" \
  --lrs3 "$CLIPS" \
  --manifest "$CLIPS/file.list" \
  --valid-ids "$CLIPS/valid_ids.txt" \
  --vocab-size 1000

# 7) Peek results
DATA_DIR=$(find "$CLIPS" -maxdepth 1 -type d -name 'data_*' | head -n1 || true)
echo "DATA_DIR=$DATA_DIR"
if [ -n "$DATA_DIR" ]; then
  echo "--- train.tsv ---"; head -n 5 "$DATA_DIR/train.tsv" || true
  echo "--- train.wrd ---"; head -n 5 "$DATA_DIR/train.wrd" || true
else
  echo "No data_* directory found (check script logs above)."
fi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 112.2 MB/s eta 0:00:00
Counts after copy:
 video: 98
 audio: 0
Sample IDs:
sa1
sa2
si1004
si1092
si1174


Traceback (most recent call last):
  File "/content/av_hubert/avhubert/preparation/count_frames.py", line 11, in <module>
    from scipy.io import wavfile
ModuleNotFoundError: No module named 'scipy'


CalledProcessError: Command 'b'set -e\n\n# 0) settings\nSPEAKER=volunteers/01M\nCLIPS="/content/drive/MyDrive/tcdtimit/${SPEAKER}/Clips"\nREPO_DIR=/content/av_hubert\n\n# 1) python deps (cv2, numpy, sentencepiece)\npip -q install "opencv-python-headless==4.8.*" "numpy==1.26.*" sentencepiece\n\n# 2) Make LRS3-like flat dirs (NO symlinks; copy files into flat video/ and audio/)\nmkdir -p "${CLIPS}/video" "${CLIPS}/audio"\n\n# Copy ROI mp4s: .../video/straightcam/*.mp4 -> .../video/<id>.mp4\nfind "${CLIPS}/video/straightcam" -maxdepth 1 -type f -name "*.mp4" -print0 | while IFS= read -r -d \'\' f; do\n  bn="$(basename "$f")"\n  [ -f "${CLIPS}/video/${bn}" ] || cp "$f" "${CLIPS}/video/${bn}"\ndone\n\n# Copy 16k wavs: .../Audio16k/Audio/straightcam/*.wav -> .../audio/<id>.wav\nfind "${CLIPS}/Audio16k/Audio/straightcam" -maxdepth 1 -type f -name "*.wav" -print0 | while IFS= read -r -d \'\' f; do\n  bn="$(basename "$f")"\n  [ -f "${CLIPS}/audio/${bn}" ] || cp "$f" "${CLIPS}/audio/${bn}"\ndone

In [ ]:
# ---- Make real lower-case audio files that match file.list, then rebuild ----
%%bash
set -e

SPEAKER=volunteers/01M
CLIPS="/content/drive/MyDrive/tcdtimit/${SPEAKER}/Clips"
REPO_DIR=/content/av_hubert

# deps
pip -q install "opencv-python-headless==4.8.*" "numpy==1.26.*" "scipy==1.11.*" sentencepiece >/dev/null
command -v ffmpeg >/dev/null 2>&1 || { apt-get update -y && apt-get install -y ffmpeg; }

# 0) ensure flat video dir (already done; safe to repeat)
mkdir -p "${CLIPS}/video"
find "${CLIPS}/video/straightcam" -maxdepth 1 -type f -iname "*.mp4" -print0 \
 | while IFS= read -r -d '' f; do
     bn="$(basename "$f")"
     [ -f "${CLIPS}/video/${bn}" ] || cp "$f" "${CLIPS}/video/${bn}"
   done

# 1) replace audio symlink with a real directory
if [ -L "${CLIPS}/audio" ] || [ ! -d "${CLIPS}/audio" ]; then
  rm -rf "${CLIPS}/audio"
  mkdir -p "${CLIPS}/audio"
fi

A16_SRC="${CLIPS}/Audio16k/Audio/straightcam"
A48_SRC="${CLIPS}/Audio/straightcam"

made=0; miss=0
while IFS= read -r id; do
  [ -z "$id" ] && continue
  out="${CLIPS}/audio/${id}.wav"
  if [ -f "$out" ]; then
    continue
  fi

  # Try to find a 16k source (any case)
  src="$(find "$A16_SRC" -maxdepth 1 -type f -iname "${id}.wav" -print -quit 2>/dev/null || true)"
  if [ -z "$src" ]; then
    # case-insensitive match by normalizing filenames
    cand="$(find "$A16_SRC" -maxdepth 1 -type f -iname "*.wav" -printf "%f\n" 2>/dev/null | awk -v t="$id.wav" 'tolower($0)==t{print; exit}')"
    [ -n "$cand" ] && src="${A16_SRC}/${cand}"
  fi

  if [ -n "$src" ]; then
    # copy (and normalize name to lower-case id.wav)
    cp "$src" "$out"
    made=$((made+1))
    continue
  fi

  # Else resample from raw Audio/straightcam
  src="$(find "$A48_SRC" -maxdepth 1 -type f -iname "${id}.wav" -print -quit 2>/dev/null || true)"
  if [ -z "$src" ]; then
    cand="$(find "$A48_SRC" -maxdepth 1 -type f -iname "*.wav" -printf "%f\n" 2>/dev/null | awk -v t="$id.wav" 'tolower($0)==t{print; exit}')"
    [ -n "$cand" ] && src="${A48_SRC}/${cand}"
  fi

  if [ -n "$src" ]; then
    ffmpeg -loglevel error -y -i "$src" -ac 1 -ar 16000 -sample_fmt s16 "$out"
    made=$((made+1))
  else
    echo "[MISS] no source WAV for ${id}"
    miss=$((miss+1))
  fi
done < "${CLIPS}/file.list"

echo "Audio prepared: made=${made}, missing=${miss}"
echo "Now under ${CLIPS}/audio (show 5):"
find "${CLIPS}/audio" -maxdepth 1 -type f -iname "*.wav" | head -n 5
echo "Counts: video=$(find "${CLIPS}/video" -maxdepth 1 -type f -iname "*.mp4" | wc -l)  audio=$(find "${CLIPS}/audio" -maxdepth 1 -type f -iname "*.wav" | wc -l)"

# 2) re-run frame counting (will create nframes.audio & nframes.video)
python "${REPO_DIR}/avhubert/preparation/count_frames.py" \
  --root "$CLIPS" \
  --manifest "$CLIPS/file.list" \
  --nshard 1 --rank 0

# 3) valid split + manifests
head -n 10 "$CLIPS/file.list" > "$CLIPS/valid_ids.txt"
python "${REPO_DIR}/avhubert/preparation/lrs3_manifest.py" \
  --lrs3 "$CLIPS" \
  --valid-ids "$CLIPS/valid_ids.txt" \
  --vocab-size 1000

# 4) peek
DATA_DIR=$(find "$CLIPS" -maxdepth 1 -type d -name 'data_*' | head -n1 || true)
echo "DATA_DIR=$DATA_DIR"
[ -n "$DATA_DIR" ] && { echo "--- train.tsv ---"; head -n 5 "$DATA_DIR/train.tsv"; echo "--- train.wrd ---"; head -n 5 "$DATA_DIR/train.wrd"; }


Audio prepared: made=98, missing=0
Now under /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/audio (show 5):
/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/audio/sa1.wav
/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/audio/sa2.wav
/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/audio/si1004.wav
/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/audio/si1092.wav
/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/audio/si1174.wav
Counts: video=98  audio=98
98 files


100%|██████████| 98/98 [00:01<00:00, 80.40it/s]
Traceback (most recent call last):
  File "/content/av_hubert/avhubert/preparation/lrs3_manifest.py", line 84, in <module>
    main()
  File "/content/av_hubert/avhubert/preparation/lrs3_manifest.py", line 27, in main
    assert os.path.isfile(nframes_audio_file) , f"{nframes_audio_file} not exist -> run count_frames.py first"
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/nframes.audio not exist -> run count_frames.py first


CalledProcessError: Command 'b'set -e\n\nSPEAKER=volunteers/01M\nCLIPS="/content/drive/MyDrive/tcdtimit/${SPEAKER}/Clips"\nREPO_DIR=/content/av_hubert\n\n# deps\npip -q install "opencv-python-headless==4.8.*" "numpy==1.26.*" "scipy==1.11.*" sentencepiece >/dev/null\ncommand -v ffmpeg >/dev/null 2>&1 || { apt-get update -y && apt-get install -y ffmpeg; }\n\n# 0) ensure flat video dir (already done; safe to repeat)\nmkdir -p "${CLIPS}/video"\nfind "${CLIPS}/video/straightcam" -maxdepth 1 -type f -iname "*.mp4" -print0 \\\n | while IFS= read -r -d \'\' f; do\n     bn="$(basename "$f")"\n     [ -f "${CLIPS}/video/${bn}" ] || cp "$f" "${CLIPS}/video/${bn}"\n   done\n\n# 1) replace audio symlink with a real directory\nif [ -L "${CLIPS}/audio" ] || [ ! -d "${CLIPS}/audio" ]; then\n  rm -rf "${CLIPS}/audio"\n  mkdir -p "${CLIPS}/audio"\nfi\n\nA16_SRC="${CLIPS}/Audio16k/Audio/straightcam"\nA48_SRC="${CLIPS}/Audio/straightcam"\n\nmade=0; miss=0\nwhile IFS= read -r id; do\n  [ -z "$id" ] && conti

In [ ]:
# Build LRS3-style manifests (train/valid) from your TCD-TIMIT folder, exactly matching your lrs3_manifest.py format.

from pathlib import Path
import os

SPEAKER = "volunteers/01M"
CLIPS   = Path(f"/content/drive/MyDrive/tcdtimit/{SPEAKER}/Clips")
VIDEO_DIR = CLIPS/"video"
AUDIO_DIR = CLIPS/"audio"
DATA_30   = CLIPS/"30h_data"
DATA_433  = CLIPS/"433h_data"
for d in (DATA_30, DATA_433): d.mkdir(exist_ok=True)

# ---- Load file.list (ids) ----
fids = [ln.strip() for ln in (CLIPS/"file.list").read_text().splitlines() if ln.strip()]
assert fids, "file.list is empty"

# ---- Load label.list (robust: supports either 'id\\ttext' or 'text-only') ----
raw_labels = [ln.rstrip("\n") for ln in (CLIPS/"label.list").read_text().splitlines()]
label_map = {}
text_only_mode = True
# detect if id\ttext format appears
for ln in raw_labels:
    if "\t" in ln:
        text_only_mode = False
        break

if text_only_mode:
    # assume same order as file.list; build map by position
    assert len(raw_labels) == len(fids), "label.list (text-only) must align 1:1 with file.list"
    label_map = {fid: raw_labels[i] for i, fid in enumerate(fids)}
else:
    # parse 'id\ttext'
    for ln in raw_labels:
        if "\t" in ln:
            i, txt = ln.split("\t", 1)
            label_map[i] = txt
        else:
            # tolerate blank lines
            pass

# ---- Load nframes (robust to either 'num' or 'id num') ----
def load_nframes(path):
    lines = [ln.strip() for ln in Path(path).read_text().splitlines() if ln.strip()]
    # detect format
    tokens0 = lines[0].split()
    if len(tokens0) == 1:
        # position-aligned with file.list
        assert len(lines) == len(fids), f"{path} must have same number of lines as file.list"
        return {fid: int(lines[i]) for i, fid in enumerate(fids)}
    else:
        # 'id num'  or 'id\t num'
        out = {}
        for ln in lines:
            parts = ln.split()
            out[parts[0]] = int(parts[-1])
        return out

nf_audio = load_nframes(CLIPS/"nframes.audio")
nf_video = load_nframes(CLIPS/"nframes.video")

# ---- Valid split ----
valid_ids_path = CLIPS/"valid_ids.txt"
valid_set = set(valid_ids_path.read_text().split()) if valid_ids_path.exists() else set()
train_all = [fid for fid in fids if fid not in valid_set]
valid     = [fid for fid in fids if fid in valid_set]
test      = []  # no dedicated test split in TCD-TIMIT layout

# ---- Checks: existence & counts ----
missing_vid = [fid for fid in fids if not (VIDEO_DIR/f"{fid}.mp4").exists()]
missing_wav = [fid for fid in fids if not (AUDIO_DIR/f"{fid}.wav").exists()]
assert not missing_vid, f"Missing videos for: {missing_vid[:5]} ..."
assert not missing_wav, f"Missing audios for: {missing_wav[:5]} ..."
assert all(fid in label_map for fid in fids), "Some IDs missing in label_map (label.list)"

# ---- Writer: EXACTLY like your lrs3_manifest.setup_target() ----
def write_split(target_dir: Path, name: str, id_list):
    tsv = target_dir/f"{name}.tsv"
    wrd = target_dir/f"{name}.wrd"
    with tsv.open("w") as fo:
        fo.write("/\n")  # your script writes '/' as the root line
        for fid in id_list:
            v_abs = (VIDEO_DIR/f"{fid}.mp4").resolve().as_posix()
            a_abs = (AUDIO_DIR/f"{fid}.wav").resolve().as_posix()
            fo.write("\t".join([fid, v_abs, a_abs, str(nf_video[fid]), str(nf_audio[fid])]) + "\n")
    with wrd.open("w") as fo:
        for fid in id_list:
            fo.write(label_map[fid].lower() + "\n")
    return tsv, wrd

# 30h_data: in LRS3 this uses 'train_sub' (trainval subset). For TCD-TIMIT we just mirror full train so downstream code doesn't break.
print("Writing 30h_data and 433h_data ...")
write_split(DATA_30,  "train", train_all)
write_split(DATA_30,  "valid", valid)
write_split(DATA_30,  "test",  test)

t_tsv, t_wrd = write_split(DATA_433, "train", train_all)
v_tsv, v_wrd = write_split(DATA_433, "valid", valid)
write_split(DATA_433, "test",  test)

print(f"433h_data -> {t_tsv} / {v_tsv}")

# ---- Build dict.wrd.txt via SentencePiece (like your script) ----
try:
    import sentencepiece as spm
    sp_dir = (CLIPS/f"spm1000"); sp_dir.mkdir(exist_ok=True)
    sp_pref = sp_dir/"spm_unigram1000"
    # Train on all labels (lowercased)
    tmp_corpus = (sp_dir/"_corpus.txt")
    tmp_corpus.write_text("\n".join(label_map[fid].lower() for fid in fids if label_map[fid].strip()) + "\n")
    spm.SentencePieceTrainer.Train(
        f'--input={tmp_corpus} --model_prefix={sp_pref} --model_type=unigram '
        f'--vocab_size=1000 --character_coverage=1.0 --unk_id=3 --bos_id=0 --eos_id=2 --pad_id=1'
    )
    # SentencePiece writes a .vocab file (tab-split piece & score). lrs3_manifest expects a plain txt (one token per line).
    vocab_txt = (sp_pref.as_posix()+".txt")
    with open(sp_pref.as_posix()+".vocab","r") as fin, open(vocab_txt,"w") as fout:
        for line in fin:
            fout.write(line.split("\t",1)[0]+"\n")
    # copy dict to both data dirs
    import shutil
    shutil.copyfile(vocab_txt, DATA_30/"dict.wrd.txt")
    shutil.copyfile(vocab_txt, DATA_433/"dict.wrd.txt")
    print("Wrote dict.wrd.txt to both data dirs.")
except Exception as e:
    print("[info] SentencePiece optional step skipped:", e)

# ---- Peek
print("--- 433h_data/train.tsv (first 5) ---")
print("\n".join((DATA_433/"train.tsv").read_text().splitlines()[:6]))
print("--- 433h_data/train.wrd (first 5) ---")
print("\n".join((DATA_433/"train.wrd").read_text().splitlines()[:5]))


Writing 30h_data and 433h_data ...
433h_data -> /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/433h_data/train.tsv / /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/433h_data/valid.tsv
[info] SentencePiece optional step skipped: Internal: src/trainer_interface.cc(664) [(trainer_spec_.vocab_size()) == (model_proto->pieces_size())] Vocabulary size too high (1000). Please set it to a value <= 542.
--- 433h_data/train.tsv (first 5) ---
/
si1540	/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/video/si1540.mp4	/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/audio/si1540.wav	151	96939
si1590	/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/video/si1590.mp4	/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/audio/si1590.wav	116	74411
si1634	/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/video/si1634.mp4	/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/audio/si1634.wav	108	68949
si1722	/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/video/si1722.mp4	/co

In [ ]:
# Robust SentencePiece vocab build with auto fallback (writes dict.wrd.txt into 30h_data and 433h_data)

from pathlib import Path
import shutil
import re

# If sentencepiece isn't available, uncomment:
# !pip -q install sentencepiece

import sentencepiece as spm

SPEAKER = "volunteers/01M"
CLIPS   = Path(f"/content/drive/MyDrive/tcdtimit/{SPEAKER}/Clips")
DATA_30 = CLIPS/"30h_data"
DATA_433= CLIPS/"433h_data"

sp_dir = CLIPS/"spm_auto"
sp_dir.mkdir(exist_ok=True)
pref = sp_dir/"spm_unigram"
corpus = sp_dir/"_corpus.txt"

# Build corpus from label.list (supports either "id<TAB>text" or "text-only")
labels = (CLIPS/"label.list").read_text().splitlines()
texts = [(ln.split("\t",1)[1] if "\t" in ln else ln).lower().strip() for ln in labels]
corpus.write_text("\n".join([t for t in texts if t])+"\n")

def train_spm(size: int):
    spm.SentencePieceTrainer.Train(
        f"--input={corpus} --model_prefix={pref} --model_type=unigram "
        f"--vocab_size={size} --character_coverage=1.0 --unk_id=3 --bos_id=0 --eos_id=2 --pad_id=1"
    )

target_size = 1000
try:
    train_spm(target_size)
except Exception as e:
    msg = str(e)
    # Parse the suggested max from the error (e.g., "<= 542")
    m = re.search(r"<=\s*(\d+)", msg)
    fallback = int(m.group(1)) if m else 512
    fallback = max(128, min(fallback, target_size))
    print(f"[info] Falling back vocab_size to {fallback}")
    train_spm(fallback)

# Convert .vocab (piece<TAB>score) → plain txt
vocab_txt = pref.as_posix()+".txt"
with open(pref.as_posix()+".vocab","r") as fin, open(vocab_txt,"w") as fout:
    for line in fin:
        fout.write(line.split("\t",1)[0]+"\n")

# Copy dict to both data dirs
shutil.copyfile(vocab_txt, DATA_30/"dict.wrd.txt")
shutil.copyfile(vocab_txt, DATA_433/"dict.wrd.txt")
print("Wrote dict.wrd.txt to:", DATA_30, "and", DATA_433)


[info] Falling back vocab_size to 542
Wrote dict.wrd.txt to: /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/30h_data and /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/433h_data


In [ ]:
# Convert train/valid .wrd -> .ltr and build dict.ltr.txt (in 433h_data)
from pathlib import Path
from collections import Counter

SPEAKER="volunteers/01M"
DATA=Path(f"/content/drive/MyDrive/tcdtimit/{SPEAKER}/Clips/433h_data")

def wrd_to_ltr_line(s: str) -> str:
    s = s.lower().strip()
    # keep a-z and apostrophes; map space -> '|'
    keep = set("abcdefghijklmnopqrstuvwxyz' ")
    s = "".join(ch for ch in s if ch in keep)
    # letters separated by spaces; '|' stands for word boundary
    return " ".join(ch if ch!=" " else "|" for ch in s)

for split in ["train","valid"]:
    wrd = (DATA/f"{split}.wrd").read_text().splitlines()
    ltr_lines = [wrd_to_ltr_line(x) for x in wrd]
    (DATA/f"{split}.ltr").write_text("\n".join(ltr_lines)+"\n")

# build dict.ltr.txt from train.ltr frequencies
cnt = Counter()
for tok in (DATA/"train.ltr").read_text().split():
    cnt[tok]+=1
with open(DATA/"dict.ltr.txt","w") as f:
    for sym,c in sorted(cnt.items(), key=lambda x:(x[0]!="|", x[0])):  # '|' first, then a..z, '
        f.write(f"{sym} {c}\n")

print("Wrote:", DATA/"train.ltr", DATA/"valid.ltr", DATA/"dict.ltr.txt")


Wrote: /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/433h_data/train.ltr /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/433h_data/valid.ltr /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/433h_data/dict.ltr.txt


In [ ]:
%%bash
set -euo pipefail
eval "$(conda shell.bash hook)"; conda activate av_hubert
export HYDRA_FULL_ERROR=1
export PYTHONPATH=/content/av_hubert/avhubert:$PYTHONPATH
export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

CKPT=/content/drive/MyDrive/AVSR_Colab/models/base_vox_iter5.pt
DATA=/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/433h_data
RUN=/content/drive/MyDrive/tcdtimit/volunteers/01M/runs/ctc_av_fix2
mkdir -p "$RUN/checkpoints"

python -m fairseq_cli.hydra_train \
  --config-dir /content/av_hubert/avhubert/conf/finetune \
  --config-name base_vox_30h \
  common.user_dir=/content/av_hubert/avhubert \
  hydra.run.dir="$RUN" \
  task.data="$DATA" \
  task.label_dir="$DATA" \
  task.labels='["ltr"]' \
  task.modalities='["audio","video"]' \
  task.is_s2s=false \
  task.normalize=true \
  task.image_aug=false \
  task.stack_order_audio=4 \
  task.max_sample_size=null \
  dataset.train_subset=train \
  dataset.valid_subset=valid \
  dataset.num_workers=1 \
  dataset.max_tokens=30000 \
  dataset.max_tokens_valid=30000 \
  model._name=av_hubert_ctc \
  model.w2v_path="$CKPT" \
  model.freeze_finetune_updates=0 \
  model.decoder_layers=2 \
  +model.w2v_args.task._name=av_hubert_pretraining \
  +model.w2v_args.task.data="$DATA" \
  +model.w2v_args.task.label_dir="$DATA" \
  +model.w2v_args.task.labels='["ltr"]' \
  +model.w2v_args.task.modalities='["audio","video"]' \
  +model.w2v_args.task.sample_rate=16000 \
  +model.w2v_args.task.label_rate=25 \
  +model.w2v_args.task.pad_audio=true \
  +model.w2v_args.task.random_crop=false \
  +model.w2v_args.task.single_target=true \
  +model.w2v_args.task.fine_tuning=false \
  +model.w2v_args.task.normalize=true \
  +model.w2v_args.task.image_aug=false \
  +model.w2v_args.task.stack_order_audio=4 \
  +model.w2v_args.label_rate=25 \
  +model.w2v_args.model._name=av_hubert \
  +model.w2v_args.model.input_modality=audio \
  +model.w2v_args.model.audio_feat_dim=104 \
  +model.w2v_args.model.label_rate=25 \
  criterion._name=ctc \
  +criterion.zero_infinity=true \
  +criterion.post_process=letter \
  optimization.max_update=150 \
  optimization.lr='[1e-4]' \
  optimization.update_freq='[4]' \
  optimization.clip_norm=5.0 \
  lr_scheduler.warmup_steps=5 \
  lr_scheduler.hold_steps=5 \
  lr_scheduler.decay_steps=140 \
  checkpoint.save_dir="$RUN/checkpoints" \
  checkpoint.save_interval_updates=50 \
  checkpoint.best_checkpoint_metric=wer \
  checkpoint.maximize_best_checkpoint_metric=false \
  distributed_training.distributed_world_size=1 \
  distributed_training.find_unused_parameters=false \
  common.log_interval=5


Process is terminated.


##Setup Directory Layout

In [ ]:
%%bash
# Make a tidy layout on your Google Drive
mkdir -p "/content/drive/MyDrive/AVSR_Colab"/{data,manifests,experiments,models,logs}

# Inside 'experiments', make a folder for your first run
mkdir -p "/content/drive/MyDrive/AVSR_Colab/experiments/baseline"


In [ ]:
# Point these at your Drive layout so later cells can reuse them
BASE = "/content/drive/MyDrive/AVSR_Colab"
DATA_DIR = f"{BASE}/data"              # where raw/augmented media will live
MANIFEST_DIR = f"{BASE}/manifests"     # where TSVs (indexes) will live
EXP_DIR = f"{BASE}/experiments"        # one subfolder per experiment
MODELS_DIR = f"{BASE}/models"          # checkpoints you already have (and new ones)
LOGS_DIR = f"{BASE}/logs"              # training logs / tensorboard etc.

print("BASE       :", BASE)
print("DATA_DIR   :", DATA_DIR)
print("MANIFESTS  :", MANIFEST_DIR)
print("EXPERIMENTS:", EXP_DIR)
print("MODELS_DIR :", MODELS_DIR)
print("LOGS_DIR   :", LOGS_DIR)


BASE       : /content/drive/MyDrive/AVSR_Colab
DATA_DIR   : /content/drive/MyDrive/AVSR_Colab/data
MANIFESTS  : /content/drive/MyDrive/AVSR_Colab/manifests
EXPERIMENTS: /content/drive/MyDrive/AVSR_Colab/experiments
MODELS_DIR : /content/drive/MyDrive/AVSR_Colab/models
LOGS_DIR   : /content/drive/MyDrive/AVSR_Colab/logs


In [ ]:
#freeze enironment (do this for each experiment so it can be replicated)
%%bash
eval "$(conda shell.bash hook)"
conda activate av_hubert

BASE="/content/drive/MyDrive/AVSR_Colab"
pip freeze > "$BASE/experiments/baseline/configs/pip_freeze.txt"
python - << 'PY' > "$BASE/experiments/baseline/configs/env_info.txt"
import torch, sys, platform
print("python:", sys.version)
print("platform:", platform.platform())
print("torch:", torch.__version__, "cuda_available:", torch.cuda.is_available())
PY
echo "Saved env snapshot."


Saved env snapshot.


## Dataset Download (very messy)

In [ ]:
#download dataset
# EDIT these two lines if you used a different filename/folder.
COOKIES = "/content/drive/MyDrive/tcd_timit/cookies.txt"
OUTDIR  = "/content/drive/MyDrive/tcd_timit"

import os, pathlib
pathlib.Path(OUTDIR).mkdir(parents=True, exist_ok=True)
print("Using cookies at:", COOKIES)
print("Saving to:", OUTDIR)

Using cookies at: /content/drive/MyDrive/tcd_timit/cookies.txt
Saving to: /content/drive/MyDrive/tcd_timit


In [ ]:
import subprocess, textwrap, os, sys

URL = "https://sigmedia.tcd.ie/tcd_timit_db/"

# Install wget (usually present, but this is safe)
!apt-get -qq update && apt-get -qq install -y wget > /dev/null

dry = f"""
wget --spider -r -np -nH --cut-dirs=1 \\
     --load-cookies "{COOKIES}" \\
     --user-agent="Mozilla/5.0" \\
     --no-check-certificate \\
     "{URL}"
"""
print(dry)
subprocess.run(dry, shell=True, check=False)


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)

wget --spider -r -np -nH --cut-dirs=1 \
     --load-cookies "/content/drive/MyDrive/tcd_timit/cookies.txt" \
     --user-agent="Mozilla/5.0" \
     --no-check-certificate \
     "https://sigmedia.tcd.ie/tcd_timit_db/"



CompletedProcess(args='\nwget --spider -r -np -nH --cut-dirs=1 \\\n     --load-cookies "/content/drive/MyDrive/tcd_timit/cookies.txt" \\\n     --user-agent="Mozilla/5.0" \\\n     --no-check-certificate \\\n     "https://sigmedia.tcd.ie/tcd_timit_db/"\n', returncode=0)

In [ ]:
%%bash
wget -nv --spider -r -np -nH --cut-dirs=1 \
     --load-cookies "/content/drive/MyDrive/tcd_timit/cookies.txt" \
     --user-agent="Mozilla/5.0" \
     --no-check-certificate \
     "https://sigmedia.tcd.ie/tcd_timit_db/" 2>&1 | tee /content/spider.log

grep -E "users/sign_in| 30[12] | 401 | 403 " /content/spider.log || echo "✅ No auth redirects detected."


  Unable to locally verify the issuer's authority.
2025-10-01 07:46:23 URL:https://sigmedia.tcd.ie/tcd_timit_db/ [38087] -> "index.html.tmp.tmp" [1]
2025-10-01 07:46:23 URL:https://sigmedia.tcd.ie/robots.txt [99/99] -> "robots.txt" [1]
2025-10-01 07:46:24 URL:https://sigmedia.tcd.ie/tcd_timit_db/Additional%20Resources [36515] -> "Additional Resources.tmp.tmp" [1]
2025-10-01 07:46:24 URL:https://sigmedia.tcd.ie/tcd_timit_db/lipspeakers [35878] -> "lipspeakers.tmp.tmp" [1]
2025-10-01 07:46:24 URL:https://sigmedia.tcd.ie/tcd_timit_db/volunteers [56732] -> "volunteers.tmp.tmp" [1]
2025-10-01 07:46:25 URL: https://sigmedia.tcd.ie/tcd_timit_db/README.txt 200 OK
unlink: No such file or directory
2025-10-01 07:46:25 URL: https://sigmedia.tcd.ie/tcd_timit_db/lipspeaker_labelfiles.mlf 200 OK
unlink: No such file or directory
2025-10-01 07:46:25 URL: https://sigmedia.tcd.ie/tcd_timit_db/sample_30_degree_video.mp4 200 OK
unlink: No such file or directory
2025-10-01 07:46:25 URL: https://sigmedia.t

In [ ]:
OUTDIR="/content/drive/MyDrive/tcdtimit"
!mkdir -p "$OUTDIR"

!wget -r -np -nH --cut-dirs=1 \
     --load-cookies "/content/drive/MyDrive/tcd_timit/cookies.txt" \
     --continue --timestamping --content-disposition \
     --wait=1 --random-wait \
     --user-agent="Mozilla/5.0" \
     --no-check-certificate \
     -P "$OUTDIR" \
     "https://sigmedia.tcd.ie/tcd_timit_db/"


--2025-10-01 07:49:38--  https://sigmedia.tcd.ie/tcd_timit_db/
Resolving sigmedia.tcd.ie (sigmedia.tcd.ie)... 134.226.112.113
Connecting to sigmedia.tcd.ie (sigmedia.tcd.ie)|134.226.112.113|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Last-modified header missing -- time-stamps turned off.
--2025-10-01 07:49:40--  https://sigmedia.tcd.ie/tcd_timit_db/
Reusing existing connection to sigmedia.tcd.ie:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/drive/MyDrive/tcdtimit/index.html’

index.html              [ <=>                ]  37.19K  --.-KB/s    in 0.1s    

2025-10-01 07:49:40 (260 KB/s) - ‘/content/drive/MyDrive/tcdtimit/index.html’ saved [38087]

Loading robots.txt; please ignore errors.
--2025-10-01 07:49:41--  https://sigmedia.tcd.ie/robots.txt
Reusing existing connection to sigmedia.tcd.ie:443.
HTTP request sent, awaitin

In [ ]:
!apt-get -qq update && apt-get -qq install -y aria2


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 126666 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/l

In [ ]:
%%bash
set -e

VOL="01M"  # change to 02M, 03F, ... later
COOKIES="/content/drive/MyDrive/tcd_timit/cookies.txt"

BASE_URL="https://sigmedia.tcd.ie/tcd_timit_db/volunteers/${VOL}/Clips"
OUT_DRIVE="/content/drive/MyDrive/tcdtimit/volunteers/${VOL}/Clips"
TMP="/content/tmpdl/volunteers/${VOL}/Clips"

#mkdir -p "$OUT_DRIVE" "$TMP"
echo "OUT_DRIVE: $OUT_DRIVE"


OUT_DRIVE: /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips


In [ ]:
%%bash
set -e
VOL="01M"
COOKIES="/content/drive/MyDrive/tcd_timit/cookies.txt"

BASE_URL="https://sigmedia.tcd.ie/tcd_timit_db/volunteers/${VOL}/Clips"
OUT_DRIVE="/content/drive/MyDrive/tcdtimit/volunteers/${VOL}/Clips"
TMP="/content/tmpdl/volunteers/${VOL}/Clips"
URL="${BASE_URL}/straightcam.zip"

#mkdir -p "$TMP" "$OUT_DRIVE"   # <- -p avoids the error

aria2c -c -x6 -s6 --file-allocation=none \
  --retry-wait=30 --max-tries=0 \
  --check-certificate=false \
  --user-agent="Mozilla/5.0" \
  --load-cookies="$COOKIES" \
  --summary-interval=15 \
  --dir="$TMP" --out="straightcam.zip" "$URL"

echo "Verifying ZIP..."
unzip -tqq "$TMP/straightcam.zip" && echo "ZIP OK"

cp -f "$TMP/straightcam.zip" "$OUT_DRIVE/straightcam.zip"
ls -lh "$OUT_DRIVE/straightcam.zip"



10/01 17:49:11 [NOTICE] Downloading 1 item(s)
[#f4e146 0B/0B CN:1 DL:0B]
[#f4e146 0B/0B CN:1 DL:0B]
[#f4e146 819KiB/0B CN:1 DL:817KiB]
[#f4e146 2.3MiB/0B CN:1 DL:1.1MiB]
[#f4e146 3.9MiB/0B CN:1 DL:1.3MiB]
[#f4e146 5.5MiB/0B CN:1 DL:1.3MiB]
[#f4e146 7.1MiB/0B CN:1 DL:1.4MiB]
[#f4e146 8.7MiB/0B CN:1 DL:1.4MiB]
[#f4e146 10MiB/0B CN:1 DL:1.4MiB]
[#f4e146 11MiB/0B CN:1 DL:1.4MiB]
[#f4e146 13MiB/0B CN:1 DL:1.4MiB]
[#f4e146 15MiB/0B CN:1 DL:1.5MiB]
[#f4e146 16MiB/0B CN:1 DL:1.5MiB]
[#f4e146 18MiB/0B CN:1 DL:1.5MiB]
 *** Download Progress Summary as of Wed Oct  1 17:49:27 2025 *** 
[#f4e146 19MiB/0B CN:1 DL:1.5MiB]
FILE: /content/tmpdl/volunteers/01M/Clips/straightcam.zip
-------------------------------------------------------------------------------

[#f4e146 19MiB/0B CN:1 DL:1.5MiB]
[#f4e146 21MiB/0B CN:1 DL:1.5MiB]
[#f4e146 22MiB/0B CN:1 DL:1.5MiB]
[#f4e146 24MiB/0B CN:1 DL:1.5MiB]
[#f4e146 26MiB/0B CN:1 DL:1.5MiB]
[#f4e146 27MiB/0B CN:1 DL:1.5MiB]
[#f4e146 29MiB/0B CN:1 DL:1.6MiB]
[#f4e14

cp: cannot stat '/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/straightcam.zip': Not a directory


CalledProcessError: Command 'b'set -e\nVOL="01M"\nCOOKIES="/content/drive/MyDrive/tcd_timit/cookies.txt"\n\nBASE_URL="https://sigmedia.tcd.ie/tcd_timit_db/volunteers/${VOL}/Clips"\nOUT_DRIVE="/content/drive/MyDrive/tcdtimit/volunteers/${VOL}/Clips"\nTMP="/content/tmpdl/volunteers/${VOL}/Clips"\nURL="${BASE_URL}/straightcam.zip"\n\n#mkdir -p "$TMP" "$OUT_DRIVE"   # <- -p avoids the error\n\naria2c -c -x6 -s6 --file-allocation=none \\\n  --retry-wait=30 --max-tries=0 \\\n  --check-certificate=false \\\n  --user-agent="Mozilla/5.0" \\\n  --load-cookies="$COOKIES" \\\n  --summary-interval=15 \\\n  --dir="$TMP" --out="straightcam.zip" "$URL"\n\necho "Verifying ZIP..."\nunzip -tqq "$TMP/straightcam.zip" && echo "ZIP OK"\n\ncp -f "$TMP/straightcam.zip" "$OUT_DRIVE/straightcam.zip"\nls -lh "$OUT_DRIVE/straightcam.zip"\n'' returned non-zero exit status 1.

In [ ]:
# paths
SRC="/content/tmpdl/volunteers/01M/Clips/straightcam.zip"
DST_DIR="/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips"
DST="$DST_DIR/straightcam.zip"

# 1) ensure destination directory exists
!mkdir -p "$DST_DIR"

# 2) copy file to Drive (source -> destination)
!cp -f "$SRC" "$DST"

# 3) confirm
!ls -lh "$DST"


mkdir: cannot create directory ‘/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips’: File exists
-rw-r--r-- 1 root root 4.8G Oct  1 18:47 /straightcam.zip


In [ ]:
SRC="/straightcam.zip"
DST_DIR="/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips"
DST="$DST_DIR/straightcam.zip"

!mkdir -p "$DST_DIR"

# progress while copying (rsync). If rsync isn't available, use cp -f "$SRC" "$DST"
!rsync -ah --progress "$SRC" "$DST"

# verify size
!ls -lh "$DST"


mkdir: cannot create directory ‘/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips’: File exists
sending incremental file list
-rw-r--r-- 1 root root 4.8G Oct  1 18:47 /straightcam.zip


In [ ]:
# 2) Create the real directory
!mkdir -p /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips


In [ ]:
# 0) Check that the source exists (should be ~4.8G)
!ls -lh /straightcam.zip
# 1) Confirm the Drive mount & the exact dest path
!ls -ld /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/


-rw-r--r-- 1 root root 4.8G Oct  1 18:47 /straightcam.zip
ls: cannot access '/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/': No such file or directory


In [ ]:
# 2) Copy the ZIP into Clips (explicit trailing slash to force "into this folder")
!cp -v /straightcam.zip /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/
!sync


'/straightcam.zip' -> '/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/straightcam.zip'


In [ ]:
# 3) Verify it’s there
!ls -lh /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/straightcam.zip


-rw------- 1 root root 4.8G Oct  1 19:04 /content/drive/MyDrive/tcdtimit/volunteers/01M/Clips/straightcam.zip


In [ ]:
# Paths
BASE="/content/drive/MyDrive/tcdtimit/volunteers/01M/Clips"
ZIP="$BASE/straightcam.zip"

# Create target folders (include camera subfolder for future-proofing)
!mkdir -p "$BASE/Video/straightcam" \
         "$BASE/Audio/straightcam" \
         "$BASE/Labels/straightcam" \
         "$BASE/Landmarks/straightcam"

# Extract each type to its bucket. -n = never overwrite; remove -n if you want overwrite.
!unzip -q -n "$ZIP" "straightcam/*.mp4" -d "$BASE/Video"
!unzip -q -n "$ZIP" "straightcam/*.wav" -d "$BASE/Audio"
!unzip -q -n "$ZIP" "straightcam/*.txt" -d "$BASE/Labels"
!unzip -q -n "$ZIP" "straightcam/*.mat" -d "$BASE/Landmarks"

# Quick glance
!echo "Labels:"    && ls -lh "$BASE/Labels/straightcam"    | head
!echo
!echo "Video:"     && ls -lh "$BASE/Video/straightcam"     | head
!echo
!echo "Audio:"     && ls -lh "$BASE/Audio/straightcam"     | head
!echo
!echo "Landmarks:" && ls -lh "$BASE/Landmarks/straightcam" | head


Labels:
total 49K
-rw------- 1 root root 180 Oct 11  1990 SA1.txt
-rw------- 1 root root 161 Oct 11  1990 SA2.txt
-rw------- 1 root root 242 Oct 11  1990 SI1004.txt
-rw------- 1 root root  94 Oct 11  1990 SI1092.txt
-rw------- 1 root root 143 Oct 11  1990 SI1174.txt
-rw------- 1 root root 145 Oct 11  1990 SI1175.txt
-rw------- 1 root root 179 Oct 11  1990 SI1178.txt
-rw------- 1 root root  92 Oct 11  1990 SI1266.txt
-rw------- 1 root root 188 Oct 11  1990 SI1447.txt

Video:
total 4.0G
-rw------- 1 root root 46M Apr 22  2014 sa1.mp4
-rw------- 1 root root 41M Apr 22  2014 sa2.mp4
-rw------- 1 root root 54M Apr 22  2014 si1004.mp4
-rw------- 1 root root 32M Apr 22  2014 si1092.mp4
-rw------- 1 root root 39M Apr 22  2014 si1174.mp4
-rw------- 1 root root 55M Apr 22  2014 si1175.mp4
-rw------- 1 root root 53M Apr 22  2014 si1178.mp4
-rw------- 1 root root 38M Apr 22  2014 si1266.mp4
-rw------- 1 root root 57M Apr 22  2014 si1447.mp4

Audio:
total 44M
-rw------- 1 root root 487K Jun 16  201